In [1604]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [1605]:
def get_data_path(df):
    if 'sample' in df.columns and 'scan_id' in df.columns:
        if len(df['sample'].unique()) == 1:
            name = df['sample'].values[0]
            scan_id_list = df['scan_id'].values
            data_path = [Path('.') / str(name) / str(scan_id) for scan_id in scan_id_list]
            return data_path
    else:
        raise ValueError('Multiple samples in metadata')

In [1606]:
def get_data(data_path):
    dat_files = list(data_path.glob('*.dat'))
    scan_id = str(data_path).split('/')[-1]
    
    if len(dat_files) != 3:
        raise ValueError('Missing data files')
    else:
        for dat_file in dat_files:
            if dat_file.name.endswith('_I.dat'):
                I_data = np.loadtxt(dat_file)
            elif dat_file.name.endswith('_Qxy.dat'):
                Qxy_data = np.loadtxt(dat_file)
            elif dat_file.name.endswith('_Qz.dat'):
                Qz_data = np.loadtxt(dat_file)
    
    return I_data, Qxy_data, Qz_data, scan_id

In [1607]:
def get_crop(box, I_data, Qxy_data, Qz_data):
    xy_min, xy_max, z_min, z_max = box
    if xy_min == None:
        xy_min = Qxy_data[0,0]
    if xy_max == None:
        xy_max = Qxy_data[-1,-1]
    if z_min == None:
        z_min = Qz_data[0,0]
    if z_max == None:
        z_max = Qz_data[-1,-1]
    mask = (Qxy_data > xy_min) & (Qxy_data < xy_max) & (Qz_data > z_min) & (Qz_data < z_max)
    I_data_copy = I_data.copy()
    I_data_copy[~mask] = np.nan
    return I_data_copy

In [1608]:
def get_avg(box, I_data, Qxy_data, Qz_data):
    crop = get_crop(box, I_data, Qxy_data, Qz_data)
    avg = np.nanmean(crop, axis=0)
    return avg

In [1609]:
def low_pass_filter(signal, window_size=None):
    if window_size is None:
        return signal
    window = np.ones(window_size) / window_size
    filtered_signal = np.convolve(signal, window, mode='valid')
    return filtered_signal

In [1610]:
metadata_path = Path('./lipid_metadata.csv').resolve()
metadata = pd.read_csv(metadata_path)

In [1611]:
azo_trans_metadata = metadata[(metadata['method']=='GIXD') & (metadata['sample']=='azo_trans')]
azo_cis_metadata = metadata[(metadata['method']=='GIXD') & (metadata['sample']=='azo_cis')]
azo_cis_2_metadata = metadata[(metadata['method']=='GIXD') & (metadata['sample']=='azo_cis_2')]
azo_cis_3_metadata = metadata[(metadata['method']=='GIXD') & (metadata['sample']=='azo_cis_3')]
water_metadata = metadata[(metadata['method']=='GIXD') & (metadata['sample']=='water')]

In [1612]:
def load_series(I_list, metadata):
    for path in get_data_path(metadata):
        try:
            I_list.append((get_data(path)[0], get_data(path)[3]))
            print("Loaded data from", path)
        except Exception as e:
            print(f"Error loading data from path {path}: {e}")

In [ ]:
I_trans_list = []
load_series(I_trans_list, azo_trans_metadata)
I_cis_list = []
load_series(I_cis_list, azo_cis_metadata)
I_cis_2_list = []
load_series(I_cis_2_list, azo_cis_2_metadata)
I_cis_3_list = []
load_series(I_cis_3_list, azo_cis_3_metadata)

In [ ]:
water_metadata = metadata[(metadata['method']=='GIXD') & (metadata['sample']=='water')]
I_water, Qxy, Qz, _ = get_data(get_data_path(water_metadata)[0])
Qxy, Qz = np.meshgrid(Qxy, Qz)
extent = [Qxy[0,0], Qxy[-1,-1], Qz[0,0], Qz[-1,-1]]
print("Qxy from", Qxy[0,0], "to", Qxy[-1,-1])
print("Qz from", Qz[0,0], "to", Qz[-1,-1])

In [1615]:
I_trans_subtracted_list = [(I_trans[0] - I_water, I_trans[1]) for I_trans in I_trans_list]
I_cis_subtracted_list = [(I_cis[0] - I_water, I_cis[1]) for I_cis in I_cis_list]
I_cis_2_subtracted_list = [(I_cis_2[0] - I_water, I_cis_2[1]) for I_cis_2 in I_cis_2_list]
I_cis_3_subtracted_list = [(I_cis_3[0] - I_water, I_cis_3[1]) for I_cis_3 in I_cis_3_list]

In [1616]:
def plot_series(I_list, extent, sample_name, figsize=None):
    fig = plt.figure(figsize=figsize)
    for i, I in enumerate(I_list):
        pressure = metadata[metadata['scan_id']==int(I[1])]['pressure_mN/m'].values[0]
        ax = fig.add_subplot(1, len(I_list), i + 1)
        ax.imshow(I[0], cmap='hot', extent=extent, origin='lower', vmin=5, vmax=15)
        ax.set_title(f'{sample_name}\n{pressure} mN/m, scan {I[1]}')
        ax.set_xticks([0.8, 1.0, 1.2, 1.4, 1.6, 1.8])
    plt.savefig(f'imshow_{sample_name}.png', dpi=100)
    plt.show()

In [ ]:
plot_series(I_trans_subtracted_list, extent, 'azo_trans', (9, 6))
plot_series(I_cis_subtracted_list, extent, 'azo_cis', (12, 6))
plot_series(I_cis_2_subtracted_list, extent, 'azo_cis_2', (6, 6))
plot_series(I_cis_3_subtracted_list, extent, 'azo_cis_3', (6, 6))

In [ ]:
step = 0.1
n_steps = int(1.2 / step)
roi_list = [
    (None, None, step * i, step * (i+1)) for i in range(n_steps)
]
avg_list = [
    get_avg(roi, I_trans_list[0][0], Qxy, Qz) for roi in roi_list
]
offset = 0
for avg in avg_list:
    plt.plot(avg/40 + offset)
    offset += 0.1
plt.show()

In [ ]:
roi = (1, 1.8, 0, 0.25)
trans_avg_list = [
    get_avg(roi, I_trans_subtracted[0], Qxy, Qz) for I_trans_subtracted in I_trans_subtracted_list
]
cis_avg_list = [
    get_avg(roi, I_cis_subtracted[0], Qxy, Qz) for I_cis_subtracted in I_cis_subtracted_list
]
cis_2_avg_list = [
    get_avg(roi, I_cis_2_subtracted[0], Qxy, Qz) for I_cis_2_subtracted in I_cis_2_subtracted_list
]
cis_3_avg_list = [
    get_avg(roi, I_cis_3_subtracted[0], Qxy, Qz) for I_cis_3_subtracted in I_cis_3_subtracted_list
]   
trans_avg_list = [low_pass_filter(trans_avg) for trans_avg in trans_avg_list]
cis_avg_list = [low_pass_filter(cis_avg) for cis_avg in cis_avg_list]
cis_2_avg_list = [low_pass_filter(cis_2_avg) for cis_2_avg in cis_2_avg_list]
cis_3_avg_list = [low_pass_filter(cis_3_avg) for cis_3_avg in cis_3_avg_list]
print(trans_avg_list[0].shape)
offset_step = 0
offset = 0
fig = plt.figure(figsize=(6, 6))
for trans_avg in trans_avg_list:
    line_trans, = plt.plot(np.linspace(roi[0], roi[1], len(trans_avg)), trans_avg/40 + offset, color='red', linewidth=1)
    offset += offset_step
offset = 0.2
for cis_avg in cis_avg_list:
    line_cis, = plt.plot(np.linspace(roi[0], roi[1], len(cis_avg)), cis_avg/40 + offset, color='blue', linewidth=1)
    offset += offset_step
offset = 0.4
for cis_2_avg in cis_2_avg_list:
    line_cis_2, = plt.plot(np.linspace(roi[0], roi[1], len(cis_2_avg)), cis_2_avg/40 + offset, color='green', linewidth=1)
    offset += offset_step
offset = 0.6
for cis_3_avg in cis_3_avg_list:
    line_cis_3, = plt.plot(np.linspace(roi[0], roi[1], len(cis_3_avg)), cis_3_avg/40 + offset, color='purple', linewidth=1)
    offset += offset_step
plt.legend([line_trans, line_cis, line_cis_2, line_cis_3], ['azo_trans', 'azo_cis', 'azo_cis_2', 'azo_cis_3'])
plt.ylim(0, 0.9)
plt.xlabel('Q_xy (A$^{-1}$)')
plt.ylabel('Intensity (a.u.)')
plt.title(f"integrated over ROI,\n Qz from {roi[2]} to {roi[3]}"+" A$^{-1}$")
plt.savefig('plot_all.png', dpi=100)
plt.show()